# Convert .root file to Pandas dataframe

In [ ]:
%load_ext autoreload
%autoreload 2
from bob import *
import pickle

Define input file name and variables to extract

In [ ]:
inputFileName = 'Hybrid_25_July_bugfixed_fullStat'

b_jets_only = False #Extract only b-jets

cols = ['jet_pt', 'jet_eta', \
         'jet_ip2d_pb','jet_ip2d_pc','jet_ip2d_pu', 'jet_ip3d_pb','jet_ip3d_pc','jet_ip3d_pu',\
         'jet_sv1_m','jet_sv1_efc','jet_sv1_ntrkv','jet_sv1_n2t', 'jet_sv1_sig3d',\
         'jet_jf_n2t','jet_jf_m','jet_sv1_Lxy','jet_sv1_L3d','jet_sv1_deltaR', 'jet_jf_nvtx1t', \
         'jet_jf_nvtx','jet_jf_ntrkAtVx','jet_jf_dRFlightDir','jet_jf_sig3d','jet_jf_efc','jet_JVT', 'jet_mv2c10',\
         'jet_LabDr_HadF',]#'jet_bH_pt'] # 'jet_dl1_pb', 'jet_dl1_pc', 'jet_dl1_pu']

Iterate over root file and save pandas df into .pkl file

compute both llr for IP2D and IP3D, apply MV2 jets preselection and sample reweighting

In [ ]:
i = 0
chunksize = int(1e6)

for df in read_root(inputFileName + '.root', 'bTag_AntiKt4EMTopoJets', columns=cols, flatten=True, chunksize=chunksize):
    tree = df
    tree = tree.drop('__array_index', 1)
    if b_jets_only:
        tree = tree[tree['jet_LabDr_HadF'] == 5]
    tree = compute_log_likelihood_ratios(tree)
    tree = jets_preselection(tree)
    tree['weights'] = sample_reweighting(tree)
    
    if i == 0:
        tree_all = tree
    else:
        tree_all.append(tree)
    i+=1

tree_all.to_pickle(inputFileName + '.pkl')